# Тестовое задание, ЕГЭLand, Рассадин Ярослав 

In [1]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta

pd.options.mode.chained_assignment = None

In [2]:
# Параметры подключения
host = 'rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net'
database = 'db1'
user = 'test_user'
password = 'j2M{CnnFq@'
port = 6432  

# Подключение к БД
try:
    conn = psycopg2.connect(host=host, database=database, user=user, password=password, port=port)
    print("Подключение успешно.")
except Exception as e:
    print("Ошибка подключения:", e)


Подключение успешно.


## Запрос к БД

In [3]:
sql_query = '''
WITH MessageTimes AS (
    SELECT
        TO_TIMESTAMP(m.created_at) AS message_time,  -- Преобразуем время сообщения из Unix в timestamp
        m.created_by,
        m.type,
        m.entity_id,
        TO_TIMESTAMP(LAG(m.created_at) OVER (PARTITION BY m.entity_id ORDER BY m.created_at)) AS previous_message_time,  -- Предыдущее сообщение
        LAG(m.type) OVER (PARTITION BY m.entity_id ORDER BY m.created_at) AS previous_message_type
    FROM
        test.chat_messages m
),
FilteredMessages AS (
    SELECT
        mt.*,
        ROW_NUMBER() OVER (PARTITION BY entity_id, 
            CASE WHEN created_by = 0 THEN 'client' ELSE 'manager' END 
            ORDER BY message_time) AS rn  -- Нумерация сообщений в рамках каждого блока
    FROM 
        MessageTimes mt
)

SELECT 
    mgr.name_mop,
    AVG(EXTRACT(EPOCH FROM response_time)) AS average_response_time_seconds,  -- Среднее время ответа в секундах
    -- Форматированный столбец времени в формате HH:MM:SS
    TO_CHAR(
        (AVG(EXTRACT(EPOCH FROM response_time)) * INTERVAL '1 second'),
        'HH24:MI:SS'
    ) AS formatted_response_time  -- Правильное преобразование в строку времени
FROM (
    SELECT 
        fm.*,
        CASE 
            WHEN (EXTRACT(HOUR FROM message_time) > 9 OR 
                  (EXTRACT(HOUR FROM message_time) = 9 AND EXTRACT(MINUTE FROM message_time) >= 30)) AND 
                  (EXTRACT(HOUR FROM previous_message_time) < 24)  -- Проверка на рабочие часы
            THEN (message_time - previous_message_time)  -- Учитываем время ответа
            WHEN message_time - previous_message_time = INTERVAL '0 seconds' THEN 
                (message_time - (DATE_TRUNC('day', previous_message_time) + INTERVAL '9 hours 30 minutes')) 
            ELSE 
                (message_time - (DATE_TRUNC('day', previous_message_time) + INTERVAL '9 hours 30 minutes'))
        END AS response_time
    FROM 
        FilteredMessages fm
    WHERE
        fm.type = 'outgoing_chat_message'  -- Запрос исходящих сообщений
        AND fm.previous_message_type = 'incoming_chat_message'  -- Предыдущее сообщение от клиента
        AND previous_message_time IS NOT NULL  -- Убедимся, что предыдущее сообщение существует
        AND rn = 1  -- Учитываем только первое сообщение из каждого блока
) AS response_times
JOIN test.managers AS mgr ON response_times.created_by = mgr.mop_id 
WHERE 
    response_time > INTERVAL '0 seconds' OR response_time = INTERVAL '0 seconds' 
GROUP BY
    mgr.mop_id, mgr.name_mop
ORDER BY
    average_response_time_seconds;

'''

try:
    df = pd.read_sql_query(sql_query, conn)
    display(df)
except Exception as e:
    print("Ошибка при выполнении запроса:", e)


,name_mop,average_response_time_seconds,formatted_response_time
0,Даша и Даша,189.461538,00:03:09
1,Порхачева Полина,389.250000,00:06:29
2,Влада и Настя,671.600000,00:11:11
3,Полина Мирзоян,742.800000,00:12:22
4,Ангелина Милованова,781.666667,00:13:01
5,Настя и Даша,1120.895833,00:18:40
6,Даша и Карина,1166.222222,00:19:26
7,Лиза и Ева,1604.757576,00:26:44
8,Ксюша и Джамиля,2831.979592,00:47:11
9,Ками и Мила,2917.661017,00:48:37


## Витрина для дашборда

In [4]:
sql_query = '''
WITH MessageTimes AS (
    SELECT
        TO_TIMESTAMP(m.created_at) AS message_time,  -- Преобразуем время сообщения из Unix в timestamp
        m.created_by,
        m.type,
        m.entity_id,
        TO_TIMESTAMP(LAG(m.created_at) OVER (PARTITION BY m.entity_id ORDER BY m.created_at)) AS previous_message_time,  -- Предыдущее сообщение
        LAG(m.type) OVER (PARTITION BY m.entity_id ORDER BY m.created_at) AS previous_message_type
    FROM
        test.chat_messages m
),
FilteredMessages AS (
    SELECT
        mt.*,
        ROW_NUMBER() OVER (PARTITION BY entity_id, 
            CASE WHEN created_by = 0 THEN 'client' ELSE 'manager' END 
            ORDER BY message_time) AS rn  -- Нумерация сообщений в рамках каждого блока
    FROM 
        MessageTimes mt
)

SELECT 
    mgr.mop_id,
    mgr.name_mop,
    rop.rop_id,
    rop.rop_name,
    DATE(message_time) AS message_date,  -- Поле для фильтрации по дням
    AVG(EXTRACT(EPOCH FROM response_time)) AS average_response_time_seconds,  -- Среднее время ответа в секундах
    -- Новый способ вычисления времени формата 'HH:MI:SS'
    TO_CHAR(
        (AVG(EXTRACT(EPOCH FROM response_time)) * INTERVAL '1 second'),
        'HH24:MI:SS'
    ) AS formatted_response_time  -- Правильное преобразование в строку времени
FROM (
    SELECT 
        fm.*,
        CASE 
            WHEN (EXTRACT(HOUR FROM message_time) > 9 OR 
                  (EXTRACT(HOUR FROM message_time) = 9 AND EXTRACT(MINUTE FROM message_time) >= 30)) AND 
                  (EXTRACT(HOUR FROM previous_message_time) < 24)  -- Проверка на рабочие часы
            THEN (message_time - previous_message_time)  -- Учитываем время ответа
            WHEN message_time - previous_message_time = INTERVAL '0 seconds' THEN 
                (message_time - (DATE_TRUNC('day', previous_message_time) + INTERVAL '9 hours 30 minutes')) 
            ELSE 
                (message_time - (DATE_TRUNC('day', previous_message_time) + INTERVAL '9 hours 30 minutes'))
        END AS response_time
    FROM 
        FilteredMessages fm
    WHERE
        fm.type = 'outgoing_chat_message'  -- Запрос исходящих сообщений
        AND fm.previous_message_type = 'incoming_chat_message'  -- Предыдущее сообщение от клиента
        AND previous_message_time IS NOT NULL  -- Убедимся, что предыдущее сообщение существует
        AND rn = 1  -- Учитываем только первое сообщение из каждого блока
) AS response_times
JOIN test.managers AS mgr ON response_times.created_by = mgr.mop_id 
JOIN test.rops AS rop ON mgr.rop_id::int = rop.rop_id  -- Приведение типа здесь
WHERE 
    response_time > INTERVAL '0 seconds' OR response_time = INTERVAL '0 seconds' 
GROUP BY
    mgr.mop_id, mgr.name_mop, rop.rop_id, rop.rop_name, message_date -- Убедитесь, что дата включена в группировку
ORDER BY
    average_response_time_seconds;

'''

try:
    df = pd.read_sql_query(sql_query, conn)
    display(df)

    df.to_csv('output.csv', index=False, encoding='utf-8-sig')
    print("Результаты успешно сохранены в 'output.csv'.")
except Exception as e:
    print("Ошибка при выполнении запроса:", e)

,mop_id,name_mop,rop_id,rop_name,message_date,average_response_time_seconds,formatted_response_time
0,6780177,Даша и Даша,2,Полина РОП,2024-10-27,189.461538,00:03:09
1,7417617,Порхачева Полина,3,Эля РОП,2024-10-27,389.250000,00:06:29
2,8558094,Алина и Юля,3,Эля РОП,2024-10-27,527.571429,00:08:47
3,11394126,Полина Мирзоян,3,Эля РОП,2024-10-26,565.500000,00:09:25
4,7996770,Лиза и Ева,2,Полина РОП,2024-10-27,608.451613,00:10:08
5,7408305,Влада и Настя,3,Эля РОП,2024-10-27,671.600000,00:11:11
6,11550890,Ангелина Милованова,3,Эля РОП,2024-10-27,781.666667,00:13:01
7,10465690,Настя и Малика,3,Эля РОП,2024-10-26,1067.433333,00:17:47
8,10465254,Настя и Даша,1,Катя РОП,2024-10-27,1120.895833,00:18:40
9,11241942,Даша и Карина,3,Эля РОП,2024-10-27,1166.222222,00:19:26


Результаты успешно сохранены в 'output.csv'.


Не удалось загрузить данные в дашборд DataLens, возникла ошибка. 

## Работа с Pandas

In [5]:
sql_query = '''
SELECT *
FROM test.chat_messages
'''

chat_messages = pd.read_sql_query(sql_query, conn)

# Преобразование колонок с временем
chat_messages.info()
chat_messages.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18929 entries, 0 to 18928
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message_id  18929 non-null  object
 1   type        18929 non-null  object
 2   entity_id   18929 non-null  int64 
 3   created_by  18929 non-null  int64 
 4   created_at  18929 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 739.5+ KB


,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199


In [6]:
sql_query = '''
SELECT *
FROM test.managers
'''

managers = pd.read_sql_query(sql_query, conn)

# Изменение типов данных
managers['rop_id'] = managers['rop_id'].astype(int)
managers.info()
managers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   mop_id    26 non-null     int64 
 1   name_mop  26 non-null     object
 2   rop_id    26 non-null     int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 648.0+ bytes


,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1
2,10262505,Вика и Марго,1
3,10262513,Ира и Варя,1
4,10465254,Настя и Даша,1


In [7]:
chat_messages['message_time'] = pd.to_datetime(chat_messages['created_at'], unit='s')

work_start = datetime.strptime("09:30", "%H:%M").time()

# Создаем предыдущие сообщения
chat_messages['previous_message_time'] = chat_messages.groupby('entity_id')['message_time'].shift(1)
chat_messages['previous_message_type'] = chat_messages.groupby('entity_id')['type'].shift(1)

# Фильтруем, чтобы получить только первые сообщения из блока
chat_messages['is_first_in_block'] = chat_messages['created_by'].ne(chat_messages['created_by'].shift()).cumsum()

# Фильтруем сообщения, учитывая блоки
chat_messages['rn'] = chat_messages.groupby(['entity_id', chat_messages['created_by'].eq(0).replace({True: 'client', False: 'manager'})])['message_time'].rank(method='first')

# Фильтруем DataFrame по условиям (только исходящие сообщения после приходящих)
response_times = chat_messages[
    (chat_messages['type'] == 'outgoing_chat_message') &
    (chat_messages['previous_message_type'] == 'incoming_chat_message') &
    (chat_messages['previous_message_time'].notnull()) &
    (chat_messages['rn'] == 1) &
    (chat_messages['is_first_in_block'] == chat_messages['is_first_in_block'])
]

# Функция для расчета времени ответа
def calculate_response_time(row):
    message_time = row['message_time']
    previous_time = row['previous_message_time']

    # Если предыдущий клиентский вопрос пришел до начала рабочего времени, устанавливаем его как 09:30
    if previous_time.time() < work_start:
        previous_time = datetime.combine(previous_time.date(), work_start)

    # Устанавливаем значение, если сообщение пришло в нерабочее время
    if message_time.time() < work_start:
        previous_time = datetime.combine(previous_time.date(), work_start)
    response_time = message_time - previous_time

    if response_time < pd.Timedelta(0):
        return pd.Timedelta(0)

    return response_time

# Применяем функцию
response_times['response_time'] = response_times.apply(calculate_response_time, axis=1)

# Среднее время
average_response = response_times.groupby('created_by').agg(
    average_response_time_seconds=('response_time', lambda x: x.dt.total_seconds().mean()),
).reset_index()

result = average_response.merge(managers, left_on='created_by', right_on='mop_id')[['name_mop', 'average_response_time_seconds']]
result['formatted_response_time'] = result['average_response_time_seconds'].apply(lambda x: str(timedelta(seconds=x)))
result = result.sort_values(by='average_response_time_seconds').reset_index().drop(columns=['index'])
display(result)


,name_mop,average_response_time_seconds,formatted_response_time
0,Даша и Даша,96.142857,0:01:36.142857
1,Софья Боднар,118.642857,0:01:58.642857
2,Даша и Карина,237.538462,0:03:57.538462
3,Полина Мирзоян,239.500000,0:03:59.500000
4,Ангелина Милованова,274.428571,0:04:34.428571
5,Порхачева Полина,286.750000,0:04:46.750000
6,Настя и Даша,321.864865,0:05:21.864865
7,Влада и Настя,522.500000,0:08:42.500000
8,Ксюша и Джамиля,540.022727,0:09:00.022727
9,Настя и Саша,977.591398,0:16:17.591398


Не проверил на всех этапах и решение не совпало, вынужден сдавать в таком виде. 

In [8]:
# Закрытие соединения
conn.close()